In [1]:
#import package
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import csv
from pyspark.sql.functions import col, create_map, lit
from itertools import chain
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.functions import array, col, explode, struct, lit,first
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType, IntegerType
from collections import Counter
from pyspark.sql.functions import date_format
from pyspark.sql.functions import collect_list
from pyspark.sql import Window
from pyspark.sql.functions import array_contains
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import struct
from pyspark.sql.window import Window
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark.sql.types import *

In [2]:
!pip install nltk
!pip install emoji

# 1.Data Preparation

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [5]:
#Read file 
# to read parquet file
venmo_t = spark.read.parquet('/FileStore/tables/VenmoSample_snappy-e020d.parquet')
venmo = venmo_t.cache()
word_t =  spark.read.format('com.databricks.spark.csv').options(header = 'true',inferschema = 'true').load('/FileStore/tables/Venmo_Word_Classification_Dictonary.csv')
word = word_t.cache()

In [6]:
emoji = spark.read.format('com.databricks.spark.csv').options(header = 'true',inferschema = 'true').load('/FileStore/tables/Venmo_Emoji_Classification_Dictionary.csv')
#Transform the Emoji Dict from wide to long
w = Window().orderBy(lit('Event'))
emoji = emoji.withColumn("row_num", row_number().over(w))

df = sc.parallelize([(1, 0.0, 0.6), (1, 0.6, 0.7)]).toDF(["A", "col_1", "col_2"])

def to_long(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("key"), col(c).alias("val")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.key", "kvs.val"])

emoji_long = to_long(emoji, ["row_num"])
emoji_long = emoji_long.drop('row_num')
emoji_long.show(5)

+--------------+----+
 key| val|
+--------------+----+
 Event|🇦🇺|
 Travel| 🏔|
 Food| 🍇|
 Activity| 👾|
Transportation| 🚄|
+--------------+----+
only showing top 5 rows

## 1.1 Text Tokenization and Cleaning

In [8]:
# tokenize the description word group
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import monotonically_increasing_id

tokenizer = Tokenizer(inputCol="description", outputCol="description_words")
tokenized_venmo = tokenizer.transform(venmo)


In [9]:
#Spilt all the Emoji into emoji_in_post section

import emoji
import re
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType

emojis_list = map(lambda x: ''.join(x.split()), emoji.UNICODE_EMOJI.keys())
escape_list = '|'.join(re.escape(p) for p in emojis_list)

def find_all_emo(plain_text):
    if plain_text is None:
        return None
    else:
        emo_list = re.findall(escape_list,plain_text)
    return emo_list

search_all_emojis = F.udf(lambda y: find_all_emo(y), ArrayType(StringType()))

tokenized_venmo = tokenized_venmo.withColumn("emoji_in_post", search_all_emojis(F.col("description")))

In [10]:
#Spilt all the word into word_in_post section
def select_text(x):
    text_list=[]
    for val in x:   
        if val.isalpha():
            text_list.append(val)
    return text_list

select_text_udf = F.udf(lambda y: select_text(y), ArrayType(StringType()))
tokenized_venmo = tokenized_venmo.withColumn("word_in_post", select_text_udf(F.col('description_words')))

In [11]:
tokenized_venmo.show(10)

+-------+-------+----------------+-------------------+---------------+-----------+--------------------+------------------+-------------+------------------+
 user1| user2|transaction_type| datetime| description|is_business| story_id| description_words|emoji_in_post| word_in_post|
+-------+-------+----------------+-------------------+---------------+-----------+--------------------+------------------+-------------+------------------+
1218774|1528945| payment|2015-11-27 10:48:19| Uber| false|5657c473cd03c9af2...| [uber]| []| [uber]|
5109483|4782303| payment|2015-06-17 11:37:04| Costco| false|5580f9702b64f70ab...| [costco]| []| [costco]|
4322148|3392963| payment|2015-06-19 07:05:31| Sweaty balls| false|55835ccb1a624b14a...| [sweaty, balls]| []| [sweaty, balls]|
 469894|1333620| charge|2016-06-03 23:34:13| 🎥| false|5751b185cd03c9af2...| [🎥]| [🎥]| []|
2960727|3442373| payment|2016-05-29 23:23:42| ⚡| false|574b178ecd03c9af2...| [⚡]| [⚡]| []|
3977544|2709470| payment|2016-09-29 22:12:07| Chipotlaid| false|57ed2f4723e064eac...| [chipotlaid]| []| [chipotlaid]|
3766386|4209061| payment|2016-05-20 10:31:15|kitchen counter| false|573e8503cd03c9af2...|[kitchen, counter]| []|[kitchen, counter]|
 730075| 804466| payment|2016-05-26 04:46:45| Food| false|57461d46cd03c9af2...| [food]| []| [food]|
5221751|4993533| payment|2016-07-14 22:53:49| Zaxby| false|5787b58d23e064eac...| [zaxby]| []| [zaxby]|
6843582|7308338| payment|2016-08-31 10:32:46| Fan sucks| false|57c64fdf23e064eac...| [fan, sucks]| []| [fan, sucks]|
+-------+-------+----------------+-------------------+---------------+-----------+--------------------+------------------+-------------+------------------+
only showing top 10 rows

In [12]:
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer

# Remove stop words
remover = StopWordsRemover(inputCol='word_in_post', outputCol='word_in_post_nostop')
tokenized_venmo_nostop = remover.transform(tokenized_venmo).select('user1','user2', 'transaction_type','datetime','description','emoji_in_post','word_in_post_nostop')

# Stem text
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
tokenized_venmo_nostop_stemmed = tokenized_venmo_nostop.withColumn("words_stemmed", stemmer_udf("word_in_post_nostop"))


In [13]:
tokenized_venmo_nostop_stemmed.show(100)

+--------+-------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
 user1| user2|transaction_type| datetime| description| emoji_in_post| word_in_post_nostop| words_stemmed|
+--------+-------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
 1218774|1528945| payment|2015-11-27 10:48:19| Uber| []| [uber]| [uber]|
 5109483|4782303| payment|2015-06-17 11:37:04| Costco| []| [costco]| [costco]|
 4322148|3392963| payment|2015-06-19 07:05:31| Sweaty balls| []| [sweaty, balls]| [sweati, ball]|
 469894|1333620| charge|2016-06-03 23:34:13| 🎥| [🎥]| []| []|
 2960727|3442373| payment|2016-05-29 23:23:42| ⚡| [⚡]| []| []|
 3977544|2709470| payment|2016-09-29 22:12:07| Chipotlaid| []| [chipotlaid]| [chipotlaid]|
 3766386|4209061| payment|2016-05-20 10:31:15| kitchen counter| []| [kitchen, counter]| [kitchen, counter]|
 730075| 804466| payment|2016-05-26 04:46:45| Food| []| [food]| [food]|
 5221751|4993533| payment|2016-07-14 22:53:49| Zaxby| []| [zaxby]| [zaxbi]|
 6843582|7308338| payment|2016-08-31 10:32:46| Fan sucks| []| [fan, sucks]| [fan, suck]|
 5317324|3942984| payment|2016-01-04 09:11:25| 👠| [👠]| []| []|
 1134661|1556430| payment|2015-10-09 01:53:52| Thanks babe| []| [thanks, babe]| [thank, babe]|
 4238868|4879587| payment|2015-10-04 08:28:01| 🍺| [🍺]| []| []|
11719500|8702716| payment|2016-07-07 21:40:39| ⛽| [⛽]| []| []|
 3625798|5692302| payment|2016-10-16 14:43:41|Hey man it's bee...| []|[hey, man, long, ...|[hey, man, long, ...|
 613908|3045405| charge|2016-05-07 06:42:17| Getaway car| []| [getaway, car]| [getaway, car]|
 4682257|1870271| payment|2016-02-24 09:14:12| 🔮 gypsy things| [🔮]| [gypsy, things]| [gypsi, thing]|
 9414481|2869012| payment|2016-04-09 09:19:46| 🔴| [🔴]| []| []|
 241386|2580543| payment|2015-05-17 06:00:19| Furniture| []| [furniture]| [furnitur]|
 656477| 656214| charge|2013-12-14 22:43:27|bed bath mostly b...| []|[bed, bath, mostl...|[bed, bath, most,...|
 7077930|2219363| payment|2015-11-03 04:44:39| Fun| []| [fun]| [fun]|
 219836| 190689| payment|2014-07-17 06:59:52| World Cup Stacks| []|[world, cup, stacks]| [world, cup, stack]|
 6222232|9644915| payment|2016-10-19 04:55:48|You're lucky we d...| []|[lucky, ask, hous...|[lucki, ask, hous...|
 1958142| 252308| charge|2015-05-31 07:33:45| Cable part 2..| []| [cable, part]| [cabl, part]|
 470775| 558040| payment|2014-07-13 01:15:28| Benny| []| [benny]| [benni]|
 801230|1387123| payment|2014-12-27 07:03:31| Blurb| []| [blurb]| [blurb]|
 5536694|4679602| payment|2016-09-17 22:54:34| Money| []| [money]| [money]|
 367859| 384908| charge|2014-01-24 18:14:30| key bees| []| [key, bees]| [key, bee]|
 8767419|8458412| payment|2016-06-10 10:59:52| A| []| []| []|
 2029081| 574846| payment|2014-11-05 06:07:54| Room| []| [room]| [room]|
 3921369|3786234| charge|2016-02-05 14:27:47| 휴지| []| [휴지]| [휴지]|
 9212366|7088535| payment|2016-09-28 07:04:15| Mas| []| [mas]| [mas]|
 7533517|6728915| payment|2016-01-24 07:40:37| Last and next week| []| [last, next, week]| [last, next, week]|
 4725068|5789825| charge|2016-03-01 08:58:44|Cheese squares ak...| []|[cheese, squares,...| [chees, squar, aka]|
 2217133|2588336| payment|2015-02-05 19:08:37| Coco| []| [coco]| [coco]|
 1309265|1594624| payment|2016-02-12 06:22:10| H2o| []| []| []|
 413268|3521812| payment|2016-04-07 01:16:50|2nd half of SF rent.| []| [half, sf]| [half, sf]|
 6530112|3087398| payment|2016-02-22 07:02:48|For bills on spli...| []| [bills]| [bill]|
 2159946| 649658| payment|2015-09-23 18:59:32| Rockies| []| [rockies]| [rocki]|
 2079232|3244153| charge|2015-08-31 05:02:30| Electric| []| [electric]| [electr]|
 1917492|3124345| payment|2015-11-16 00:41:45| XL| []| [xl]| [xl]|
 1391021|1702448| payment|2016-04-03 10:00:11| Spider rolls| []| [spider, rolls]| [spider, roll]|
 1556129|2908933| payment|2016-02-09 10:26:11| Nutella| []| [nutella]| [nutella]|
 2281104|2221219| payment|2

Word Cout

In [15]:
nword = word.drop('_c9')

#Collect all the word category into list
People = [row[0] for row in nword.select(nword.columns[0]).collect()]
Food = [row[0] for row in nword.select(nword.columns[1]).collect()]
Event = [row[0] for row in nword.select(nword.columns[2]).collect()]
Activity = [row[0] for row in nword.select(nword.columns[3]).collect()]
Travel = [row[0] for row in nword.select(nword.columns[4]).collect()]
Tran = [row[0] for row in nword.select(nword.columns[5]).collect()]
Utility = [row[0] for row in nword.select(nword.columns[6]).collect()]
Cash = [row[0] for row in nword.select(nword.columns[7]).collect()]
Illegal = [row[0] for row in nword.select(nword.columns[8]).collect()]

In [16]:
#Compare the word with the word category dict
def word_type(x):
    type = []
    for val in x:
        if val in People:
            type.append('People')
        elif val in Food:
            type.append("Food")
        elif val in Event:
            type.append("Event")
        elif val in Activity:
            type.append("Activity")
        elif val in Utility:
            type.append("Utility")
        elif val in Cash:
            type.append("Cash")
        elif val in Illegal:
            type.append("Illegal")
        elif val in Tran:
            type.append("Transportation")   
        else: type.append("Others")
    return list(set(type))

In [17]:
#Build the UDF function
FindWordCate_udf = F.udf(lambda y: word_type(y), ArrayType(StringType()))

#Run the UDF function
venmo_cleaned= tokenized_venmo_nostop_stemmed.withColumn("word_type", FindWordCate_udf(F.col('words_stemmed')))

#show result
venmo_cleaned.sort('user1').show()

+-----+-------+----------------+-------------------+--------------------+-------------+--------------------+--------------------+------------------+
user1| user2|transaction_type| datetime| description|emoji_in_post| word_in_post_nostop| words_stemmed| word_type|
+-----+-------+----------------+-------------------+--------------------+-------------+--------------------+--------------------+------------------+
 2| 220| payment|2012-11-23 06:03:42|Grab that couch. ...| []| [grab, good, deal]| [grab, good, deal]| [Others]|
 3|2382556| payment|2016-10-06 10:49:45|http://ense.nyc/e...| []| []| []| []|
 3|1204190| payment|2016-10-09 01:56:24|Can I borrow thos...| []|[borrow, sunglass...|[borrow, sunglass...| [Others, Food]|
 3| 52| payment|2016-09-22 15:30:09|Hehe.. we need so...| []|[need, hear, like...|[need, hear, like...| [Others, People]|
 3|2382556| payment|2016-10-07 08:50:23| Keepin' it real| []| [real]| [real]| [Others]|
 3|7854140| payment|2016-10-09 03:36:13|Check out this re...| []|[check, recording...|[check, record, m...| [Others, Utility]|
 3|1079020| payment|2016-10-07 23:37:56|Good luck on your...| []|[good, luck, firs...|[good, luck, firs...| [Others]|
 4|9271982| payment|2016-03-03 12:45:57| 🏕| [🏕]| []| []| []|
 4| 968271| payment|2014-02-04 06:51:33| for bball league| []| [bball, league]| [bball, leagu]|[Others, Activity]|
 4| 125527| payment|2012-12-15 05:51:12| BBQ| []| [bbq]| [bbq]| [Food]|
 4| 221578| payment|2016-04-17 03:35:09| 🔫| [🔫]| []| []| []|
 4| 187560| charge|2015-06-17 09:23:30| samovar| []| [samovar]| [samovar]| [Others]|
 4| 122744| payment|2012-12-03 03:35:53| hot sex| []| [hot, sex]| [hot, sex]| [Food, Illegal]|
 10| 43| payment|2013-03-03 12:47:10| Food| []| [food]| [food]| [Food]|
 10| 71056| payment|2013-04-05 09:48:56| And it begins!| []| []| []| []|
 10| 43| payment|2012-12-31 12:44:12| Everything tonight| []|[everything, toni...| [everyth, tonight]| [Others]|
 10| 43| payment|2012-12-23 11:08:45| Grub| []| [grub]| [grub]| [Food]|
 10| 255| payment|2012-11-25 09:20:39| Thank you bro!| []| [thank]| [thank]| [People]|
 10| 36523| payment|2015-01-21 23:29:21|That cheesecake ....| []| [cheesecake]| [cheesecak]| [Others]|
 10| 255| payment|2013-01-14 08:18:43| Sweater| []| [sweater]| [sweater]| [Activity]|
+-----+-------+----------------+-------------------+--------------------+-------------+--------------------+--------------------+------------------+
only showing top 20 rows

Emoji Cout

In [19]:
emoji = sqlContext.read.format('com.databricks.spark.csv').options(header = 'true',inferschema = 'true').load('/FileStore/tables/Venmo_Emoji_Classification_Dictionary.csv')
#Collect all the emoji category into list
Event_e = [row[0] for row in emoji.select(emoji.columns[0]).collect()]
Travel_e = [row[0] for row in emoji.select(emoji.columns[1]).collect()]
Food_e = [row[0] for row in emoji.select(emoji.columns[2]).collect()]
Activity_e = [row[0] for row in emoji.select(emoji.columns[3]).collect()]
Tran_e = [row[0] for row in emoji.select(emoji.columns[4]).collect()]
People_e = [row[0] for row in emoji.select(emoji.columns[5]).collect()]
Utility_e = [row[0] for row in emoji.select(emoji.columns[6]).collect()]

In [20]:
#Compare the emoji with the emoji category dict
def emoji_type(x):
    type = []
    for val in x:
        if val in Event_e:
            type.append('Event')
        elif val in Travel_e:
            type.append('Travel')
        elif val in Activity_e:
            type.append('Activity')
        elif val in Tran_e:
            type.append('Transportation')
        elif val in People_e:
            type.append('People')
        elif val in Utility_e:
            type.append('Utility')
        else: type.append("Others")
    if type is []:
        type.append("Others")
    return list(set(type))

In [21]:
#Build the UDF function
FindEmoCateg_udf = F.udf(lambda y: emoji_type(y), ArrayType(StringType()))

#Run the UDF function
venmo_cleaned = venmo_cleaned.withColumn("emoji_type", FindEmoCateg_udf(F.col('emoji_in_post')))

Concate Word Count and Emoji count

In [23]:
#Concate emoji type and word type into a new col called type
from pyspark.sql.functions import concat
col_list = ['word_type','emoji_type']
venmo_type_t = venmo_cleaned.withColumn('type',concat(*col_list))
venmo_type_final = venmo_type_t.cache()

In [24]:
# save the result datafrmae into parquet file, so that later we can use it more efficiently
venmo_type_final.write.parquet("venmo_type_final.parquet", mode='overwrite')

In [25]:
venmo_type_final.sort('user2','user1').show()

+-------+-----+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
 user1|user2|transaction_type| datetime| description| emoji_in_post| word_in_post_nostop| words_stemmed| word_type| emoji_type| type|
+-------+-----+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
 43| 2| charge|2016-09-07 07:53:16| The crack up| []| [crack]| [crack]| [Illegal]| []| [Illegal]|
 43| 2| charge|2016-04-09 09:29:31| Headphones 🎹| [🎹]| [headphones]| [headphon]| [Others]| [Others]| [Others, Others]|
 191142| 2| payment|2016-09-26 09:56:55| All the food 😊| [😊]| [food]| [food]| [Food]| [People]| [Food, People]|
 43| 3| payment|2016-06-27 01:14:37| Boom-a-chick-boom| []| []| []| []| []| []|
 263437| 3| payment|2016-07-22 21:16:37|Birthdays come on...| []|[birthdays, come,...|[birthday, come, ...|[Others, People, ...| []|[Others, People, ...|
 567957| 3| payment|2016-10-29 09:28:07| Kottu| []| [kottu]| [kottu]| [Others]| []| [Others]|
 125527| 4| payment|2012-12-10 10:27:55| Parking at tcw.| []| [parking]| [park]| [Activity]| []| [Activity]|
 125755| 4| payment|2013-01-04 10:20:36| straight cash homie| []|[straight, cash, ...|[straight, cash, ...| [Others]| []| [Others]|
 676003| 6| payment|2014-06-28 00:18:16| Funsies| []| [funsies]| [funsi]| [Others]| []| [Others]|
 676003| 6| payment|2015-08-27 00:48:57| 😱| [😱]| []| []| []| [People]| [People]|
 688883| 6| payment|2016-05-02 23:47:32| 🍻 🍻 and 🌶 🌶| [🍻, 🍻, 🌶, 🌶]| []| []| []| [Others]| [Others]|
4330489| 6| payment|2015-04-14 04:34:23|Hidden valley boa...| []|[hidden, valley, ...|[hidden, valley, ...|[Others, Food, Ac...| []|[Others, Food, Ac...|
 659067| 8| payment|2016-02-10 11:55:25| B+/A-| []| []| []| []| []| []|
 900433| 8| charge|2016-03-02 07:35:34| Insta🍏🍎🍐🍊🍋|[🍏, 🍎, 🍐, 🍊, 🍋]| []| []| []| [Others]| [Others]|
 900433| 8| charge|2016-05-17 01:18:05| Food| []| [food]| [food]| [Food]| []| [Food]|
 900433| 8| payment|2015-09-20 10:49:54| Uber| []| [uber]| [uber]| [Transportation]| []| [Transportation]|
 900433| 8| charge|2015-08-11 02:08:47| Costco| []| [costco]| [costco]| [Food]| []| [Food]|
 900433| 8| charge|2016-04-18 05:46:42| 🐀🔫 💡| [🐀, 🔫, 💡]| []| []| []|[Others, Utility]| [Others, Utility]|
 243| 9| charge|2012-06-28 04:28:32|Avicii here we co...| []| [avicii]| [avicii]| [Others]| []| [Others]|
 243| 9| charge|2013-10-03 08:43:46| TP| []| [tp]| [tp]| [Utility]| []| [Utility]|
+-------+-----+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
only showing top 20 rows

# 2. EDA

## 2.1 What is the percent of emoji only transactions?

In [28]:
#write udf to return False when the 'word_only' list is not null
def emoji_only(value):
    if (value == []) or (value == ['']):
        return 'True'
    else: return 'False'

In [29]:
# Run the udf function
emoji_only_test = F.udf(lambda x: emoji_only(x))
venmo_new = venmo_type_final.withColumn('emoji_only',emoji_only_test(F.col('words_stemmed')))

In [30]:
# calculate the percentage
total = venmo_new.count()
percent = venmo_new.groupBy("emoji_only").count().withColumnRenamed("count","per_per_group").withColumn("per_total",(F.col("per_per_group")/total)*100)

In [31]:
# show the result
percent.show()

+----------+-------------+-----------------+
emoji_only|per_per_group| per_total|
+----------+-------------+-----------------+
 False| 4847232|68.14478618927204|
 True| 2265905|31.85521381072795|
+----------+-------------+-----------------+

## 2.2 Which are the top 5 most popular emoji?

In [33]:
venmo_new.select("emoji_in_post").withColumn("emoji",F.explode(F.col("emoji_in_post"))).groupBy("emoji").count().sort(F.col("count").desc()).show(5)

+-----+------+
emoji| count|
+-----+------+
 🍕|215039|
 🍻|145233|
 💸|124727|
 🍷|111157|
 🎉| 94327|
+-----+------+
only showing top 5 rows

## 2.3 Which are the top three most popular emoji categories?

In [35]:
venmo_new.select("emoji_type").withColumn("emoji_type_count",F.explode(F.col("emoji_type"))).groupBy("emoji_type_count").count().sort(F.col("count").desc()).show(3)

+----------------+-------+
emoji_type_count| count|
+----------------+-------+
 Others|1740575|
 People| 652442|
 Activity| 284399|
+----------------+-------+
only showing top 3 rows

## 2.4 Spending Behavior Profile

In [37]:
venmo_type_final.createOrReplaceTempView("venmo_type_final")

# since we only want the profile for transaction where user spent money, we only select user1 when transaction type is payment 
# and user2 when transaction type is charge
venmo_spending = spark.sql("""(SELECT user1 as user, type, datetime FROM venmo_type_final WHERE transaction_type = 'payment') UNION ALL (SELECT user2 as user, type, datetime FROM venmo_type_final WHERE transaction_type = 'charge') """)

category_count = venmo_spending.select('user', F.explode('type').alias("type"))\
                                       .groupBy('user','type')\
                                       .count()\
                                       .select('user','type', 'count',F.sum('count').over(Window.partitionBy("user")).alias('total_count'))\
                                       .sort('user', 'type')
category_count.show() # 'count' column shows the count of transaction for certain user in the whole lifetime and 'total_count' is number of all transaction for the certain user

+----+--------+-----+-----------+
user| type|count|total_count|
+----+--------+-----+-----------+
 2| Illegal| 1| 4|
 2| Others| 3| 4|
 3| Food| 1| 8|
 3| Others| 5| 8|
 3| People| 1| 8|
 3| Utility| 1| 8|
 4|Activity| 1| 7|
 4| Food| 2| 7|
 4| Illegal| 1| 7|
 4| Others| 2| 7|
 4| Travel| 1| 7|
 8| Food| 2| 5|
 8| Others| 2| 5|
 8| Utility| 1| 5|
 9| Food| 1| 7|
 9| Others| 3| 7|
 9| Utility| 3| 7|
 10|Activity| 1| 14|
 10| Food| 5| 14|
 10| Others| 6| 14|
+----+--------+-----+-----------+
only showing top 20 rows

In [38]:
# percentage = 'count'/'total_count'
category_count = category_count.select('user','type',round(((col("count") /col("total_count"))),2).alias("proportion"))

In [39]:
# pivot the percentage of each category
category_count = category_count.groupBy('user')\
                      .pivot("type").sum("proportion").fillna(0).sort("user")


Dynamic spending profile for each user

In [41]:
# create the month indicator for each transaction
# We set the first transaction happend in time 0, and any transaction happend in 30 days after first transaction tagged as 'month 1', and so on. thus here we use ceil() function

window = Window.partitionBy('user').orderBy('datetime')
venmo_spending_dyn = venmo_spending.withColumn("month_passed", F.ceil((F.datediff(venmo.datetime, 
                                   first(venmo.datetime).over(window)))/30))

In [42]:
# Save as parquet format
venmo_spending_dyn.coalesce(1).write.mode('overwrite').format("parquet").save('/FileStore/tables/venmo_spending_dyn.parquet')

In [43]:
# First, count each users' trasnaction count for each category in each month.
venmo_spending_dyn = spark.read.parquet('/FileStore/tables/venmo_spending_dyn.parquet')
user_dynamic_profile = venmo_spending_dyn.select('user','month_passed', F.explode('type').alias("type"))\
                                       .groupBy('user','month_passed','type')\
                                       .count()\
                                       .select('user','month_passed','type', 'count',F.sum('count').over(Window.partitionBy("user","month_passed")).alias('total_count'))\
                                       .sort('user','month_passed', 'type')

In [44]:
# pivor the transaction count 
user_dynamic_profile = user_dynamic_profile.groupBy('user','month_passed',"total_count")\
                       .pivot("type").sum("count").fillna(0).sort("user","month_passed")
user_dynamic_profile.show(20)

+----+------------+-----------+--------+----+-----+----+-------+------+------+--------------+------+-------+
user|month_passed|total_count|Activity|Cash|Event|Food|Illegal|Others|People|Transportation|Travel|Utility|
+----+------------+-----------+--------+----+-----+----+-------+------+------+--------------+------+-------+
 2| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
 2| 42| 2| 0| 0| 0| 0| 0| 2| 0| 0| 0| 0|
 2| 47| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 3| 0| 2| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 3| 1| 6| 0| 0| 0| 1| 0| 4| 0| 0| 0| 1|
 4| 0| 2| 0| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 4| 1| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
 4| 15| 2| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0|
 4| 40| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0|
 4| 42| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
 8| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
 8| 7| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
 8| 9| 2| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1|
 8| 10| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
 9| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
 9| 2| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
 9| 5| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 9| 7| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 9| 8| 2| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0|
 9| 16| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
+----+------------+-----------+--------+----+-----+----+-------+------+------+--------------+------+-------+
only showing top 20 rows

In [45]:
# Since users may not have transaction in each month of the 12-month lifetime, left join the entire month period for each user

venmo_spending.createOrReplaceTempView("venmo_spending")
month_cycle = spark.sql("""SELECT distinct v.user,sub.month
             FROM venmo_spending v CROSS JOIN (select 0 as month union all select 1 as month union all select 2 union all select 3 union all select 4 union all select 5 union all select 6 union all select 7 union all select 8 union all select 9 union all select 10 union all select 11 union all select 12) as sub  
            ORDER BY v.user,sub.month""")

user_dynamic_profile.createOrReplaceTempView("user_dynamic_profile")
month_cycle.createOrReplaceTempView("month_cycle")

accum_profile = spark.sql("""
                SELECT l.user, l.month, ifnull(r.total_count,0) as total_count
                      ,ifnull(r.Activity,0) as Activity, ifnull(r.Cash,0) as Cash, ifnull(r.Event,0) as Event, ifnull(r.Food,0) as Food, ifnull(r.Illegal,0) as Illegal, 
                      ifnull(r.Others,0) as Others, ifnull(r.People,0) as People, ifnull(r.Transportation,0) as Transportation, ifnull(r.Travel,0) as Travel,
                      ifnull(r.Utility,0) as Utility
                FROM month_cycle l LEFT JOIN user_dynamic_profile r
                ON l.user = r.user AND l.month = r.month_passed """)

In [46]:
#calculate the proportion for each month
accum_profile.createOrReplaceTempView("accum_profile")

accum_profile_prop = spark.sql(
         """SELECT user, month,
            ifnull( round(Activity/ total_count ,2),0) as activity_perc,
            ifnull( round(Food/ total_count ,2),0) as food_perc,
            ifnull( round(Illegal/ total_count ,2),0) as illegal_perc,
             ifnull( round(Others/ total_count ,2),0) as others_perc,
             ifnull( round(People/ total_count ,2),0) as people_perc,
             ifnull( round(Utility/ total_count ,2),0)  as utility_perc,
            ifnull( round(Cash/ total_count ,2),0) as cash_perc,
            ifnull( round(Event/ total_count ,2),0)  as event_perc,
            ifnull( round(Transportation/ total_count ,2),0)  as transp_perc,
             ifnull( round(Travel/ total_count ,2),0)  as travel_perc
            FROM accum_profile
            ORDER BY user, month """    
                 )

In [47]:
# write the result file to parquet
accum_profile_prop.coalesce(1).write.mode('overwrite').format("parquet").save('/FileStore/tables/accum_profile_prop.parquet')

In [48]:
# For the last step, we need to calculate the mean and s.d. for transaction proportion within each category for each month interval

# first thing is to unpivot the result percentage
accum_profile_prop = spark.read.parquet('/FileStore/tables/accum_profile_prop.parquet')
accum_profile_prop.createOrReplaceTempView("accum_profile_prop")
accum_profile_unpivot = spark.sql("""SELECT user,month,
             stack(10, 'activity_perc',activity_perc,
                    'food_perc', food_perc, 
                    'illegal_perc', illegal_perc,
                    'others_perc',others_perc, 
                    'people_perc',people_perc, 
                    'utility_perc',utility_perc, 
                    'cash_perc',cash_perc, 
                    'event_perc',event_perc, 
                    'transp_perc',transp_perc, 
                    'travel_perc',travel_perc) as (category,proportion)
            FROM accum_profile_prop""")

# then calculate mean and standard deviation
accum_profile_unpivot.createOrReplaceTempView("accum_profile_unpivot")
accum_profile_sum = spark.sql("""SELECT month,category,
                          mean(proportion) as mean,
                          2*stddev(proportion) as sd_2
                          FROM accum_profile_unpivot
                          GROUP BY month, category
                          ORDER BY month, category 
                          """)

In [49]:
accum_profile_sum.sort('category','month').show(130) # display the result to see if things make sense

+-----+-------------+--------------------+--------------------+
month| category| mean| sd_2|
+-----+-------------+--------------------+--------------------+
 0|activity_perc| 0.07116119311892026| 0.43431396452781607|
 1|activity_perc| 0.00777948100198242| 0.14668594612474226|
 2|activity_perc|0.006914521375216575| 0.1387047447040055|
 3|activity_perc|0.006584658348843189| 0.13489223368255548|
 4|activity_perc|0.006215307309886318| 0.1310673752640688|
 5|activity_perc|0.006285755846022932| 0.13179495803500105|
 6|activity_perc|0.006179720501204542| 0.13122047219704103|
 7|activity_perc|0.005933686651032136| 0.12831577461883956|
 8|activity_perc|0.005490679814806171| 0.12333682350305325|
 9|activity_perc|0.005076004526641...| 0.1187168284830836|
 10|activity_perc|0.004740185158165491| 0.1144644073793422|
 11|activity_perc|0.004537638388202...| 0.11164479889900722|
 12|activity_perc|0.004466246801635484| 0.11120164470713059|
 0| cash_perc|0.004478315178129...| 0.10238199454020758|
 1| cash_perc|5.459016451781683E-4| 0.03438398595625175|
 2| cash_perc|4.833622772710663E-4| 0.03258249779626655|
 3| cash_perc|4.493635299267325...| 0.03138978335183969|
 4| cash_perc|4.235808870367977...| 0.03021447087225499|
 5| cash_perc|3.976914837208189...|0.029293128980014224|
 6| cash_perc|4.012234744824833E-4|0.029416391467039244|
 7| cash_perc|3.857787995145995E-4|0.028943392423037905|
 8| cash_perc|3.482391647064288...| 0.02741726339612651|
 9| cash_perc|3.292536022747108E-4|0.026682124540282483|
 10| cash_perc|3.202545880293111...| 0.02647953632122144|
 11| cash_perc|3.134708526243505E-4|0.026134523086756754|
 12| cash_perc|2.787603691063797E-4| 0.02430103744031549|
 0| event_perc| 0.01820439639435171| 0.22012222473851598|
 1| event_perc|0.002055654210097345| 0.07372085760054774|
 2| event_perc|0.001857302235207...| 0.06972286941475418|
 3| event_perc|0.001804015437557...| 0.06897308535773485|
 4| event_perc|0.001732752853172342| 0.06774696969356352|
 5| event_perc|0.001671783753198...| 0.06619444546128866|
 6| event_perc|0.001666432386842803| 0.06650337820601175|
 7| event_perc|0.001539658829265...| 0.06333206539846732|
 8| event_perc|0.001488996558755...| 0.06247423724390116|
 9| event_perc|0.001371942648299...|0.059734882717931405|
 10| event_perc|0.001236543737967189| 0.05701719768380433|
 11| event_perc|0.001196419611178...| 0.0560212130652923|
 12| event_perc|0.001140010071066...|0.054460381993645494|
 0| food_perc| 0.07685547396288754| 0.47337615720688014|
 1| food_perc|0.010643280498786187| 0.17875053056220128|
 2| food_perc| 0.00931274310238767| 0.1676746810434449|
 3| food_perc|0.008893761810372824| 0.1636576543803421|
 4| food_perc|0.008390644405932046| 0.15898246354081463|
 5| food_perc|0.008364327960911877| 0.15875517626222613|
 6| food_perc|0.008256931420661369| 0.15773282625885593|
 7| food_perc|0.007950110852909302| 0.1539934517338952|
 8| food_perc|0.007389169332711768| 0.1486485533269579|
 9| food_perc|0.006716654270594934| 0.14177116745692328|
 10| food_perc|0.006424989234990327| 0.13881815449625748|
 11| food_perc|0.006221490517894787| 0.13617215683458353|
 12| food_perc| 0.00609269296057333| 0.13495903542743998|
 0| illegal_perc|0.027828537417758863| 0.2746299708222209|
 1| illegal_perc|0.003686401924534507| 0.09866448385565588|
 2| illegal_perc| 0.00317739935160825| 0.09127500627980625|
 3| illegal_perc|0.002982726163065791| 0.08847804975513777|
 4| illegal_perc|0.002826918217954...| 0.0861825597239348|
 5| illegal_perc|0.002757434974003757| 0.08514701938648306|
 6| illegal_perc|0.002665794493297...| 0.08305171550123817|
 7| illegal_perc|0.002490493873730...| 0.08051825369973616|
 8| illegal_perc|0.002290157934256...| 0.07724862457101278|
 9| illegal_perc|0.002039662387945984| 0.07271493623357152|
 10| illegal_perc|0.001985864029921342| 0.07208449456899658|
 11| illegal_perc|0.001824562371220...| 0.06875083030379936|
 12| illegal_perc|0.001714665857659...| 0.06659109077382375|
 0| others_perc| 0.5817837709917121| 0.8407

In [50]:
# to plot by matplotlib, we need to convert pyspark DF to pandas DataFrame
accum_profile_sum_pd = accum_profile_sum.toPandas()

# draw the mean and standard deviation in y, and month in x-axis. There are one line for each category.
fig, ax = plt.subplots(figsize=(20,10))    # 1

for key, group in accum_profile_sum_pd.groupby('category'):
    group.plot('month', 'mean', yerr='sd_2', label=key, ax=ax)   # 2

plt.show()
# we can see that the category 'other' is higher than other lines, indicating that most of the description includes category 'others' which are not included in our current dictionary.

# 3. Social Network Analysis

In [52]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import csv
from pyspark.sql.functions import col, create_map, lit
from itertools import chain
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.functions import array, col, explode, struct, lit
from pyspark.sql.functions import array_contains
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import struct
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
# to read parquet file
venmo_data = spark.read.parquet('/FileStore/tables/VenmoSample_snappy-e020d.parquet')
venmo_data.sort('user1').show()

+-----+-------+----------------+-------------------+--------------------+-----------+--------------------+
user1| user2|transaction_type| datetime| description|is_business| story_id|
+-----+-------+----------------+-------------------+--------------------+-----------+--------------------+
 2| 220| payment|2012-11-23 06:03:42|Grab that couch. ...| false|54e419e6cd03c9af2...|
 3|2382556| payment|2016-10-06 10:49:45|http://ense.nyc/e...| false|57f5c9d923e064eac...|
 3|1204190| payment|2016-10-09 01:56:24|Can I borrow thos...| false|57f9415823e064eac...|
 3| 52| payment|2016-09-22 15:30:09|Hehe.. we need so...| false|57e3969123e064eac...|
 3|2382556| payment|2016-10-07 08:50:23| Keepin' it real| false|57f6ff6023e064eac...|
 3|7854140| payment|2016-10-09 03:36:13|Check out this re...| false|57f958bd23e064eac...|
 3|1079020| payment|2016-10-07 23:37:56|Good luck on your...| false|57f7cf6423e064eac...|
 4|9271982| payment|2016-03-03 12:45:57| 🏕| false|56d7c185cd03c9af2...|
 4| 968271| payment|2014-02-04 06:51:33| for bball league| false|52f01d75c62cecac3...|
 4| 125527| payment|2012-12-15 05:51:12| BBQ| false|50cb9fed25ee44b9a...|
 4| 221578| payment|2016-04-17 03:35:09| 🔫| false|5712a1fdcd03c9af2...|
 4| 187560| charge|2015-06-17 09:23:30| samovar| false|5580da2211615c78a...|
 4| 122744| payment|2012-12-03 03:35:53| hot sex| false|54e41a2ecd03c9af2...|
 10| 43| payment|2013-03-03 12:47:10| Food| false|5132d71225ee44b9a...|
 10| 71056| payment|2013-04-05 09:48:56| And it begins!| false|515e3d2125ee44b9a...|
 10| 43| payment|2012-12-31 12:44:12| Everything tonight| false|50e118db25ee44b9a...|
 10| 43| payment|2012-12-23 11:08:45| Grub| false|50d6766b25ee44b9a...|
 10| 255| payment|2012-11-25 09:20:39| Thank you bro!| false|50b172d925ee44b9a...|
 10| 36523| payment|2015-01-21 23:29:21|That cheesecake ....| false|54bfc5d1ca134c404...|
 10| 255| payment|2013-01-14 08:18:43| Sweater| false|50f34fc025ee44b9a...|
+-----+-------+----------------+-------------------+--------------------+-----------+--------------------+
only showing top 20 rows

In [54]:
# create connection edges by union both user1-user2 and user2-user1
venmo_data.createOrReplaceTempView('venmo_data')

friending_t = spark.sql("SELECT distinct user, friend\
                               FROM  \
                               (SELECT user1 as user, user2 as friend\
                               FROM venmo_data \
                               UNION \
                               SELECT user2 as user, user1 as friend\
                               FROM venmo_data) as sub")
friending = friending_t.cache()
friending.createOrReplaceTempView('friending')
friending.sort('user','friend').show()

+----+-------+
user| friend|
+----+-------+
 2| 43|
 2| 220|
 2| 191142|
 3| 43|
 3| 52|
 3| 263437|
 3| 567957|
 3|1079020|
 3|1204190|
 3|2382556|
 3|7854140|
 4| 122744|
 4| 125527|
 4| 125755|
 4| 187560|
 4| 221578|
 4| 968271|
 4|9271982|
 6| 676003|
 6| 688883|
+----+-------+
only showing top 20 rows

## 3.1 Friends and Friends of Friends

#### - First method: try running sql in spark

In [57]:
# try running sql in spark
# for each user, find the friend of original friend, and the fof can not be in original friend of user
fof =  spark.sql("SELECT distinct l.user, l.friend, r.friend as fof, 1 as connect\
                          FROM friending l LEFT JOIN friending r \
                          ON l.friend = r.user \
                          WHERE r.friend NOT IN (SELECT friend \
                                                 FROM friending \
                                                 WHERE user = l.user)\
                          AND r.friend !=l.user \
                          ORDER BY l.user, l.friend, fof")

In [58]:
spark.sql("SELECT distinct l.user, l.friend, r.friend as fof, 1 as connect\
                          FROM friending l LEFT JOIN friending r \
                          ON l.friend = r.user \
                          WHERE r.friend NOT IN (SELECT friend \
                                                 FROM friending \
                                                 WHERE user = l.user)\
                          AND r.friend !=l.user \
                          ORDER BY l.user, l.friend, fof").explain()

== Physical Plan ==
Sort [user#52 ASC NULLS FIRST, friend#53 ASC NULLS FIRST, fof#104 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(user#52 ASC NULLS FIRST, friend#53 ASC NULLS FIRST, fof#104 ASC NULLS FIRST, 200), [id=#317]
 +- *(5) HashAggregate(keys=[user#52, friend#53, fof#104, 1 AS 1#158], functions=[])
 +- *(5) Project [user#52, friend#53, friend#108 AS fof#104]
 +- SortMergeJoin [user#52], [user#52#144], LeftAnti, ((friend#108 = friend#53#143) || isnull(friend#53#143))
 :- Sort [user#52 ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(user#52, 200), [id=#307]
 : +- *(3) Project [user#52, friend#53, friend#108]
 : +- *(3) SortMergeJoin [friend#53], [user#107], Inner, NOT (friend#108 = user#52)
 : :- Sort [friend#53 ASC NULLS FIRST], false, 0
 : : +- Exchange hashpartitioning(friend#53, 200), [id=#297]
 : : +- *(1) Filter (isnotnull(user#52) && isnotnull(friend#53))
 : : +- *(1) InMemoryTableScan [user#52, friend#53], [isnotnull(user#52), isnotnull(friend#53)]
 : : +- InMemoryRelation [user#52, friend#53], StorageLevel(disk, memory, deserialized, 1 replicas)
 : : +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
 : : +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
 : : +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
 : : +- Union
 : : :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
 : : : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
 : : +- *(2) Project [user2#17 AS user#54, user1#16 AS friend#55]
 : : +- *(2) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
 : +- Sort [user#107 ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(user#107, 200), [id=#301]
 : +- *(2) Filter (isnotnull(friend#108) && isnotnull(user#107))
 : +- *(2) InMemoryTableScan [user#107, friend#108], [isnotnull(friend#108), isnotnull(user#107)]
 : +- InMemoryRelation [user#107, friend#108], StorageLevel(disk, memory, deserialized, 1 replicas)
 : +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
 : +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
 : +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
 : +- Union
 : :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
 : : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
 : +- *(2) Project [user2#17 AS user#54, user1#16 AS friend#55]
 : +- *(2) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
 +- Sort [user#52#144 ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(user#52#144, 200), [id=#310]
 +- *(4) Project [friend#53 AS friend#53#143, user#52 AS user#52#144]
 +- *(4) Filter isnotnull(user#52)
 +- *(4) InMemoryTableScan [friend#53, user#52], [isnotnull(user#52)]
 +- InMemoryRelation [user#52, friend#53], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
 +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
 +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
 +- Union
 :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
 : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:

#### - Second Method: try spark-specific sql-liked join

In [60]:
friending_copy = friending

#join - normal join
friending.alias("user").\
         join(friending_copy.alias("friend"), col("user.friend") == col("friend.user"),"left_outer").\
         filter(col('user.user')!=col('friend.friend')).explain()

== Physical Plan ==
*(3) SortMergeJoin [friend#53], [user#195], Inner, NOT (user#52 = friend#196)
:- Sort [friend#53 ASC NULLS FIRST], false, 0
: +- Exchange hashpartitioning(friend#53, 200), [id=#416]
: +- *(1) Filter (isnotnull(user#52) && isnotnull(friend#53))
: +- *(1) InMemoryTableScan [user#52, friend#53], [isnotnull(user#52), isnotnull(friend#53)]
: +- InMemoryRelation [user#52, friend#53], StorageLevel(disk, memory, deserialized, 1 replicas)
: +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
: +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
: +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
: +- Union
: :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
: : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
: +- *(2) Project [user2#17 AS user#54, user1#16 AS friend#55]
: +- *(2) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
+- Sort [user#195 ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(user#195, 200), [id=#420]
 +- *(2) Filter (isnotnull(friend#196) && isnotnull(user#195))
 +- *(2) InMemoryTableScan [user#195, friend#196], [isnotnull(friend#196), isnotnull(user#195)]
 +- InMemoryRelation [user#195, friend#196], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
 +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
 +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
 +- Union
 :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
 : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
 +- *(2) Project [user2#17 AS user#54, user1#16 AS friend#55]
 +- *(2) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>

#### - Third, try broadcast join that specific to Spark

In [62]:
# When the table is small enough to fit into the memory of a single worker node, with some
# breathing room of course, we can optimize our join. Although we can use a big table–to–big
# table communication strategy, it can often be more efficient to use a broadcast join.
friending.alias("user").\
        join(broadcast(friending_copy.alias("friend")), col("user.friend") == col("friend.user"),"left_outer").\
        filter(col('user.user')!=col('friend.friend')).explain()

== Physical Plan ==
*(2) BroadcastHashJoin [friend#53], [user#257], Inner, BuildRight, NOT (user#52 = friend#258)
:- *(2) Filter (isnotnull(user#52) && isnotnull(friend#53))
: +- *(2) InMemoryTableScan [user#52, friend#53], [isnotnull(user#52), isnotnull(friend#53)]
: +- InMemoryRelation [user#52, friend#53], StorageLevel(disk, memory, deserialized, 1 replicas)
: +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
: +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
: +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
: +- Union
: :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
: : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
: +- *(2) Project [user2#17 AS user#54, user1#16 AS friend#55]
: +- *(2) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint))), [id=#470]
 +- *(1) Filter (isnotnull(friend#258) && isnotnull(user#257))
 +- *(1) InMemoryTableScan [user#257, friend#258], [isnotnull(friend#258), isnotnull(user#257)]
 +- InMemoryRelation [user#257, friend#258], StorageLevel(disk, memory, deserialized, 1 replicas)
 +- *(4) HashAggregate(keys=[user#52, friend#53], functions=[])
 +- Exchange hashpartitioning(user#52, friend#53, 200), [id=#106]
 +- *(3) HashAggregate(keys=[user#52, friend#53], functions=[])
 +- Union
 :- *(1) Project [user1#16 AS user#52, user2#17 AS friend#53]
 : +- *(1) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>
 +- *(2) Project [user2#17 AS user#54, user1#16 AS friend#55]
 +- *(2) FileScan parquet [user1#16,user2#17] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/FileStore/tables/VenmoSample_snappy-e020d.parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user1:int,user2:int>

In [63]:
fof.show(100)

+----+-------+--------+-------+
user| friend| fof|connect|
+----+-------+--------+-------+
 2| 43| 3| 1|
 2| 43| 10| 1|
 2| 43| 19| 1|
 2| 43| 96| 1|
 2| 43| 629| 1|
 2| 43| 3565| 1|
 2| 43| 20530| 1|
 2| 43| 20639| 1|
 2| 43| 47104| 1|
 2| 43| 49778| 1|
 2| 43| 55895| 1|
 2| 43| 72106| 1|
 2| 43| 82697| 1|
 2| 43| 183628| 1|
 2| 43| 276439| 1|
 2| 43| 297967| 1|
 2| 43| 307603| 1|
 2| 43| 316862| 1|
 2| 43| 1491816| 1|
 2| 191142| 613769| 1|
 2| 191142| 851628| 1|
 2| 191142| 892097| 1|
 3| 43| 2| 1|
 3| 43| 10| 1|
 3| 43| 19| 1|
 3| 43| 96| 1|
 3| 43| 629| 1|
 3| 43| 3565| 1|
 3| 43| 20530| 1|
 3| 43| 20639| 1|
 3| 43| 47104| 1|
 3| 43| 49778| 1|
 3| 43| 55895| 1|
 3| 43| 72106| 1|
 3| 43| 82697| 1|
 3| 43| 183628| 1|
 3| 43| 276439| 1|
 3| 43| 297967| 1|
 3| 43| 307603| 1|
 3| 43| 316862| 1|
 3| 43| 1491816| 1|
 3| 52| 225| 1|
 3| 52| 59208| 1|
 3| 263437| 117793| 1|
 3| 263437| 141210| 1|
 3| 263437| 147891| 1|
 3| 263437| 148035| 1|
 3| 263437| 148049| 1|
 3| 263437| 171598| 1|
 3| 263437| 263468| 1|
 3| 263437| 266763| 1|
 3| 263437| 277545| 1|
 3| 263437| 331257| 1|
 3| 263437| 472894| 1|
 3| 263437| 525358| 1|
 3| 263437| 679648| 1|
 3| 263437| 809443| 1|
 3| 263437| 1327569| 1|
 3| 567957| 281136| 1|
 3| 567957| 393813| 1|
 3| 567957| 521279| 1|
 3| 567957| 565575| 1|
 3| 567957| 581123| 1|
 3| 567957| 615598| 1|
 3| 567957| 855452| 1|
 3| 567957| 1028243| 1|
 3| 567957| 1092849| 1|
 3| 567957| 1114544| 1|
 3| 567957| 1375797| 1|
 3| 567957| 2047571| 1|
 3| 567957| 2210989| 1|
 3| 567957| 3135797| 1|
 3| 567957| 4153462| 1|
 3| 567957| 4694550| 1|
 3|1079020| 4376570| 1|
 3|1079020|14884139| 1|
 3|1204190| 53860| 1|
 3|1204190| 173334| 1|
 3|1204190| 374536| 1|
 3|1204190| 655009| 1|
 3|1204190| 1107915| 1|
 3|1204190| 1354887| 1|
 3|1204190| 1400679| 1|
 3|1204190| 1416613| 1|
 3|1204190| 1708995| 1|
 3|1204190| 1745178| 1|
 3|1204190| 1804420| 1|
 3|1204190| 2087306| 1|
 3|1204190| 2110371| 1|
 3|1204190| 2266248| 1|
 3|1204190| 4402869| 1|
 3|1204190| 5561315| 1|
 4| 125527| 159614| 1|
 4| 187560| 65679| 1|
 4| 187560| 3247825| 1|
 4| 221578| 2405| 1|
 4| 221578| 114204| 1|
 4| 221578| 664326| 1|
 4| 221578| 1746662| 1|
 4| 968271| 1598403| 1|
+----+-------+--------+-------+
only showing top 100 rows

Build a user's friends and friends of friends dynamically

In [65]:
venmo_data.createOrReplaceTempView('venmo_data')

dynamic_friending = spark.sql("SELECT distinct user, friend, datetime\
                               FROM  \
                               (SELECT user1 as user, user2 as friend, datetime \
                               FROM venmo_data \
                               UNION \
                               SELECT user2 as user, user1 as friend, datetime \
                               FROM venmo_data) ")
dynamic_friending.createOrReplaceTempView('dynamic_friending')
dynamic_friending.sort('user','friend').show()

+----+-------+-------------------+
user| friend| datetime|
+----+-------+-------------------+
 2| 43|2016-09-07 07:53:16|
 2| 43|2016-04-09 09:29:31|
 2| 220|2012-11-23 06:03:42|
 2| 191142|2016-09-26 09:56:55|
 3| 43|2016-06-27 01:14:37|
 3| 52|2016-09-22 15:30:09|
 3| 263437|2016-07-22 21:16:37|
 3| 567957|2016-10-29 09:28:07|
 3|1079020|2016-10-07 23:37:56|
 3|1204190|2016-10-09 01:56:24|
 3|2382556|2016-10-06 10:49:45|
 3|2382556|2016-10-07 08:50:23|
 3|7854140|2016-10-09 03:36:13|
 4| 122744|2012-12-03 03:35:53|
 4| 125527|2012-12-10 10:27:55|
 4| 125527|2012-12-15 05:51:12|
 4| 125755|2013-01-04 10:20:36|
 4| 187560|2015-06-17 09:23:30|
 4| 221578|2016-04-17 03:35:09|
 4| 968271|2014-02-04 06:51:33|
+----+-------+-------------------+
only showing top 20 rows

In [66]:
# create the month indicator for each transaction
window = Window.partitionBy('user').orderBy('datetime')
dynamic_friending = dynamic_friending.withColumn("month_passed", F.ceil((F.datediff(dynamic_friending.datetime, 
                                   first(dynamic_friending.datetime).over(window)))/30) )
dynamic_friending.sort('month_passed','user','friend').show()

+----+-------+-------------------+------------+
user| friend| datetime|month_passed|
+----+-------+-------------------+------------+
 2| 220|2012-11-23 06:03:42| 0|
 3| 43|2016-06-27 01:14:37| 0|
 4| 122744|2012-12-03 03:35:53| 0|
 6| 676003|2014-06-28 00:18:16| 0|
 8| 900433|2015-08-11 02:08:47| 0|
 9| 243|2012-06-28 04:28:32| 0|
 10| 255|2012-11-25 09:20:39| 0|
 11| 54724|2012-05-15 22:08:58| 0|
 12| 112|2012-09-01 03:40:16| 0|
 13| 126820|2012-09-28 06:11:39| 0|
 16|1457073|2014-07-12 04:20:53| 0|
 18| 334955|2016-02-21 07:17:22| 0|
 19| 43|2012-05-26 02:19:04| 0|
 24| 704870|2013-12-20 10:31:39| 0|
 28| 769787|2014-05-22 11:25:51| 0|
 29| 231463|2014-04-25 23:37:38| 0|
 31| 192|2013-05-19 11:19:15| 0|
 32| 42|2012-11-07 01:29:50| 0|
 34| 5114|2014-07-21 23:31:20| 0|
 42| 11|2012-08-25 00:30:23| 0|
+----+-------+-------------------+------------+
only showing top 20 rows

In [67]:
dynamic_degree = dynamic_friending.groupBy('user','month_passed').agg(countDistinct('friend').alias('degree')).sort('user','month_passed')
dynamic_degree.show()

+----+------------+------+
user|month_passed|degree|
+----+------------+------+
 2| 0| 1|
 2| 42| 1|
 2| 47| 2|
 3| 0| 1|
 3| 1| 1|
 3| 3| 1|
 3| 4| 4|
 3| 5| 1|
 4| 0| 1|
 4| 1| 1|
 4| 2| 1|
 4| 15| 1|
 4| 31| 1|
 4| 40| 1|
 4| 42| 1|
 6| 0| 1|
 6| 10| 1|
 6| 15| 1|
 6| 23| 1|
 8| 0| 1|
+----+------------+------+
only showing top 20 rows

In [68]:
# for each user, find the friend of original friend, and the fof can not be in original friend of user
dynamic_friending.createOrReplaceTempView('dynamic_friending')

dynamic_fof =  spark.sql("SELECT distinct l.user, l.friend, l.month_passed, r.friend as fof \
                          FROM dynamic_friending l LEFT JOIN dynamic_friending r \
                          ON l.friend = r.user AND l.month_passed = r.month_passed AND r.friend !=l.user\
                          WHERE r.friend NOT IN (SELECT friend \
                                                 FROM dynamic_friending \
                                                 WHERE user = l.user)\
                          ORDER BY l.month_passed, l.user, l.friend, fof")
dynamic_fof.sort('user', 'month_passed','friend').show(100)

+----+-------+------------+--------+
user| friend|month_passed| fof|
+----+-------+------------+--------+
 3| 43| 0| 20530|
 3| 263437| 1| 266763|
 3| 567957| 5| 1375797|
 3| 567957| 5| 281136|
 3| 567957| 5| 615598|
 8| 900433| 2| 2734056|
 8| 900433| 10| 704469|
 9| 243| 16| 63980|
 10| 255| 0| 66916|
 10| 43| 2| 47104|
 10| 43| 2| 19|
 10| 43| 4| 47104|
 10| 43| 4| 55895|
 10| 43| 11| 49778|
 10| 43| 18| 629|
 11| 57391| 1| 66374|
 11| 42| 4| 28332|
 11| 171286| 18| 195662|
 11| 171286| 18| 1591530|
 11| 171286| 18| 402124|
 11| 50225| 20| 855754|
 11| 119906| 23| 1943126|
 11| 119906| 23| 1221103|
 11| 119906| 23| 117793|
 11| 119906| 23| 862946|
 12| 112| 0| 54724|
 12| 221813| 22| 4598683|
 12| 221813| 22| 459250|
 12| 295034| 22| 222155|
 13| 126820| 0| 142831|
 13| 11| 6| 133784|
 13| 11| 18| 41798|
 13| 11| 18| 171286|
 13| 11| 18| 146271|
 16| 747| 3| 1009|
 16| 747| 3| 158173|
 18| 334955| 0| 217385|
 19| 43| 0| 20530|
 19| 43| 18| 10|
 19| 43| 18| 629|
 19| 85471| 22| 665589|
 24| 704870| 0| 525006|
 29| 231463| 0| 299|
 32| 42| 0| 11|
 34| 5114| 0| 267449|
 42| 11| 0| 54724|
 42| 11| 22| 13|
 42| 11| 22| 183628|
 43| 49778| 7| 11|
 43| 49778| 7| 150620|
 43| 10| 9| 133032|
 43| 47104| 19| 295569|
 43| 276439| 27| 2443779|
 52| 225| 14| 37578|
 52| 59208| 46| 308088|
 52| 59208| 46| 1658586|
 52| 59208| 46| 4748154|
 52| 59208| 46| 1950330|
 52| 59208| 46|10609050|
 58| 540365| 0| 91906|
 58| 436320| 2| 2060461|
 58| 179463| 3| 1839838|
 58| 179463| 3| 785405|
 58| 179463| 3| 432037|
 58| 179463| 3| 2530010|
 58|4345709| 9| 343808|
 96| 43| 0| 20530|
 96| 43| 12| 47104|
 108| 717814| 0| 2582444|
 112| 54724| 0| 11|
 112| 221813| 21| 4598683|
 126|1009401| 23| 1289355|
 129| 243198| 0| 161092|
 130| 66870| 0| 1009|
 130| 69382| 13| 510479|
 133| 71454| 0| 30746|
 134| 42442| 0| 2024|
 142| 735596| 0| 1509001|
 145| 581918| 0| 1283931|
 145| 581918| 1| 1283931|
 149| 36464| 0| 28899|
 156| 12| 16| 112|
 158|2345090| 0| 688965|
 160| 516084| 15| 251784|
 164| 429597| 0| 490961|
 169| 243010| 0| 222713|
 177| 952922| 22| 578|
 180| 569936| 0| 788109|
 187|2517467| 0| 1169267|
 211|2450651| 0| 2739808|
 212| 713796| 0| 809059|
 213| 160635| 13| 349947|
 213| 160635| 13| 112446|
 214| 183586| 0| 712094|
 214| 460593| 24| 267302|
 214| 460593| 24| 236567|
 214|1365216| 25| 719471|
 224| 531663| 0| 499691|
 224|3161682| 4| 3618654|
 224|3161682| 4| 458068|
+----+-------+------------+--------+
only showing top 100 rows

In [69]:
# Calculate friends of friends degree
dynamic_fofdegree = dynamic_fof.groupBy('user','month_passed').agg(countDistinct('fof').alias('fof_degree'))

In [70]:
dynamic_fofdegree.createOrReplaceTempView('dynamic_fofdegree')
dynamic_degree.createOrReplaceTempView('dynamic_degree')

dynamic_degree_fofdegree = spark.sql('SELECT l.user, l.month_passed, l.degree, ifnull(r.fof_degree,0) as fof_degree\
                                     FROM dynamic_degree l LEFT JOIN dynamic_fofdegree r \
                                     ON l.user =r.user AND l.month_passed = r.month_passed \
                                     WHERE l.month_passed <= 12')

In [71]:
dynamic_degree_fofdegree.sort('user','month_passed').show()

+----+------------+------+----------+
user|month_passed|degree|fof_degree|
+----+------------+------+----------+
 2| 0| 1| 0|
 3| 0| 1| 1|
 3| 1| 1| 1|
 3| 3| 1| 0|
 3| 4| 4| 0|
 3| 5| 1| 3|
 4| 0| 1| 0|
 4| 1| 1| 0|
 4| 2| 1| 0|
 6| 0| 1| 0|
 6| 10| 1| 0|
 8| 0| 1| 0|
 8| 2| 1| 1|
 8| 7| 2| 0|
 8| 9| 1| 0|
 8| 10| 1| 1|
 9| 0| 1| 0|
 9| 2| 1| 0|
 9| 5| 1| 0|
 9| 7| 1| 0|
+----+------------+------+----------+
only showing top 20 rows

In [72]:
# confirm how many user are in the dataframe
dynamic_degree.select(countDistinct("user")).show()

+--------------------+
count(DISTINCT user)|
+--------------------+
 3018657|
+--------------------+

In [73]:
# export the result set 
dynamic_degree_fofdegree.coalesce(1).write.mode('overwrite').format("parquet").save('/FileStore/tables/dynamic_fof_degree.parquet')
dynamic_friending.coalesce(1).write.mode('overwrite').format("parquet").save('/FileStore/tables/dynamic_friending.parquet')
dynamic_degree.coalesce(1).write.mode('overwrite').format("parquet").save('/FileStore/tables/dynamic_degree.parquet')

## 3.2 Dynamic Clustering Coefficient

In [75]:
venmo = spark.read.parquet('/FileStore/tables/VenmoSample_snappy-e020d.parquet')
venmo.show(100)

+--------+-------+----------------+-------------------+--------------------+-----------+--------------------+
 user1| user2|transaction_type| datetime| description|is_business| story_id|
+--------+-------+----------------+-------------------+--------------------+-----------+--------------------+
 1218774|1528945| payment|2015-11-27 10:48:19| Uber| false|5657c473cd03c9af2...|
 5109483|4782303| payment|2015-06-17 11:37:04| Costco| false|5580f9702b64f70ab...|
 4322148|3392963| payment|2015-06-19 07:05:31| Sweaty balls| false|55835ccb1a624b14a...|
 469894|1333620| charge|2016-06-03 23:34:13| 🎥| false|5751b185cd03c9af2...|
 2960727|3442373| payment|2016-05-29 23:23:42| ⚡| false|574b178ecd03c9af2...|
 3977544|2709470| payment|2016-09-29 22:12:07| Chipotlaid| false|57ed2f4723e064eac...|
 3766386|4209061| payment|2016-05-20 10:31:15| kitchen counter| false|573e8503cd03c9af2...|
 730075| 804466| payment|2016-05-26 04:46:45| Food| false|57461d46cd03c9af2...|
 5221751|4993533| payment|2016-07-14 22:53:49| Zaxby| false|5787b58d23e064eac...|
 6843582|7308338| payment|2016-08-31 10:32:46| Fan sucks| false|57c64fdf23e064eac...|
 5317324|3942984| payment|2016-01-04 09:11:25| 👠| false|5689c6bdcd03c9af2...|
 1134661|1556430| payment|2015-10-09 01:53:52| Thanks babe| false|5616bbc0cd03c9af2...|
 4238868|4879587| payment|2015-10-04 08:28:01| 🍺| false|561080a1cd03c9af2...|
11719500|8702716| payment|2016-07-07 21:40:39| ⛽| false|577e69e723e064eac...|
 3625798|5692302| payment|2016-10-16 14:43:41|Hey man it's bee...| false|58032fad23e064eac...|
 613908|3045405| charge|2016-05-07 06:42:17| Getaway car| false|572d2bd9cd03c9af2...|
 4682257|1870271| payment|2016-02-24 09:14:12| 🔮 gypsy things| false|56cd03e4cd03c9af2...|
 9414481|2869012| payment|2016-04-09 09:19:46| 🔴| false|570866c2cd03c9af2...|
 241386|2580543| payment|2015-05-17 06:00:19| Furniture| false|5557cc0407f81c33e...|
 656477| 656214| charge|2013-12-14 22:43:27|bed bath mostly b...| false|52ac6e93d56b6bac5...|
 7077930|2219363| payment|2015-11-03 04:44:39| Fun| false|5637cb37cd03c9af2...|
 219836| 190689| payment|2014-07-17 06:59:52| World Cup Stacks| false|53c711f87d0b0354e...|
 6222232|9644915| payment|2016-10-19 04:55:48|You're lucky we d...| false|58069a6423e064eac...|
 1958142| 252308| charge|2015-05-31 07:33:45| Cable part 2..| false|556a56e94b9f431ba...|
 470775| 558040| payment|2014-07-13 01:15:28| Benny| false|53c17b407d0b0354e...|
 801230|1387123| payment|2014-12-27 07:03:31| Blurb| false|549de943ca134c404...|
 5536694|4679602| payment|2016-09-17 22:54:34| Money| false|57dd673a23e064eac...|
 367859| 384908| charge|2014-01-24 18:14:30| key bees| false|52e23d06c62cecac3...|
 8767419|8458412| payment|2016-06-10 10:59:52| A| false|575a3b38cd03c9af2...|
 2029081| 574846| payment|2014-11-05 06:07:54| Room| false|54594e3a7d0b0354e...|
 3921369|3786234| charge|2016-02-05 14:27:47| 휴지| false|56b440e3cd03c9af2...|
 9212366|7088535| payment|2016-09-28 07:04:15| Mas| false|57eb08ff23e064eac...|
 7533517|6728915| payment|2016-01-24 07:40:37| Last and next week| false|56a40f75cd03c9af2...|
 4725068|5789825| charge|2016-03-01 08:58:44|Cheese squares ak...| false|56d4e944cd03c9af2...|
 2217133|2588336| payment|2015-02-05 19:08:37| Coco| false|54d34f3591bd05aa9...|
 1309265|1594624| payment|2016-02-12 06:22:10| H2o| false|56bd0992cd03c9af2...|
 413268|3521812| payment|2016-04-07 01:16:50|2nd half of SF rent.| false|57055292cd03c9af2...|
 6530112|3087398| payment|2016-02-22 07:02:48|For bills on spli...| false|56ca4218cd03c9af2...|
 2159946| 649658| payment|2015-09-23 18:59:32| Rockies| false|56029424cd03c9af2...|
 2079232|3244153| charge|2015-08-31 05:02:30| Electric| false|55e37d76cd03c9af2...|
 1917492|3124345| payment|2015-11-16 00:41:45| XL| false|5648b5c9cd03c9af2...|
 1391021|1702448| payment|2016-04-03 10:00:11| Spider rolls| false|5700873bcd03c9af2...|
 1556129|2908933| payment|2016-02-09 10:26:11| Nutella| false|56b94e43cd03c9af2...|
 2281104|2221219| payment|2015-06-06 03:05:28| DAT CPK

In [76]:
dynamic_friending = spark.read.parquet('/FileStore/tables/dynamic_friending.parquet')
dynamic_friending.createOrReplaceTempView('f1')
dynamic_friending.createOrReplaceTempView('f2')
dynamic_friending.createOrReplaceTempView('f3')

dyn_triangle_t = spark.sql("""SELECT distinct f1.month_passed, f1.user as tri_1, f1.friend as tri_2 
                                 ,f2.friend as tri_3, f3.friend as back_tri_1
                          FROM f1 
                          LEFT JOIN f2 ON f1.month_passed = f2.month_passed AND f1.friend = f2.user AND f1.user !=f2.friend
                          LEFT JOIN f3 ON f2.month_passed = f3.month_passed AND f2.friend = f3.user AND f3.friend = f1.user  
                          ORDER BY f1.month_passed, tri_1,  tri_2, tri_3""")
dyn_triangle = dyn_triangle_t.cache()

In [77]:
dyn_triangle.createOrReplaceTempView('dyn_triangle')

count_trian_t = spark.sql("""SELECT month_passed, tri_1 as user , count(*)/2 as tri_count
                          FROM dyn_triangle
                          WHERE tri_2 is not null AND tri_3 is not null and back_tri_1 is not null
                          GROUP BY month_passed,user
                          ORDER BY user, month_passed""")
count_trian = count_trian_t.cache()

In [78]:
count_trian.createOrReplaceTempView('count_trian')
dynamic_degree = spark.read.parquet('/FileStore/tables/dynamic_degree.parquet')
dynamic_degree.createOrReplaceTempView('dynamic_degree')

trian_triple = spark.sql("""SELECT d.month_passed, d.user, d.degree, ifnull(c.tri_count,0) as tri_count
                            FROM dynamic_degree d LEFT JOIN count_trian c
                            ON d.month_passed = c.month_passed AND d.user = c.user
                            WHERE d.month_passed <13
                            """
                            )
trian_triple.sort('month_passed','user').show()

+------------+----+------+---------+
month_passed|user|degree|tri_count|
+------------+----+------+---------+
 0| 2| 1| 0.0|
 0| 3| 1| 0.0|
 0| 4| 1| 0.0|
 0| 6| 1| 0.0|
 0| 8| 1| 0.0|
 0| 9| 1| 0.0|
 0| 10| 1| 0.0|
 0| 11| 1| 0.0|
 0| 12| 1| 0.0|
 0| 13| 1| 0.0|
 0| 16| 1| 0.0|
 0| 18| 1| 0.0|
 0| 19| 1| 0.0|
 0| 24| 1| 0.0|
 0| 28| 1| 0.0|
 0| 29| 1| 0.0|
 0| 31| 1| 0.0|
 0| 32| 1| 0.0|
 0| 34| 1| 0.0|
 0| 42| 1| 0.0|
+------------+----+------+---------+
only showing top 20 rows

In [79]:
# calculate triple count for each user by factorial:
# the potential triple count for a user who has n friend: nC2
triple_count = trian_triple\
             .withColumn('triple_count',F.when(col('degree')>=2,factorial(col('degree'))/(2*factorial(col('degree')-2))).otherwise(0))

In [80]:
cluster_coef = triple_count.withColumn('cc',F.when(col("triple_count")>0, round(col('tri_count')/col('triple_count'),2)).otherwise(0))
cluster_coef.show(20)

+------------+----+------+---------+------------+---+
month_passed|user|degree|tri_count|triple_count| cc|
+------------+----+------+---------+------------+---+
 0| 2| 1| 0.0| 0.0|0.0|
 0| 3| 1| 0.0| 0.0|0.0|
 1| 3| 1| 0.0| 0.0|0.0|
 3| 3| 1| 0.0| 0.0|0.0|
 4| 3| 4| 0.0| 6.0|0.0|
 5| 3| 1| 0.0| 0.0|0.0|
 0| 4| 1| 0.0| 0.0|0.0|
 1| 4| 1| 0.0| 0.0|0.0|
 2| 4| 1| 0.0| 0.0|0.0|
 0| 6| 1| 0.0| 0.0|0.0|
 10| 6| 1| 0.0| 0.0|0.0|
 0| 8| 1| 0.0| 0.0|0.0|
 2| 8| 1| 0.0| 0.0|0.0|
 7| 8| 2| 0.0| 1.0|0.0|
 9| 8| 1| 0.0| 0.0|0.0|
 10| 8| 1| 0.0| 0.0|0.0|
 0| 9| 1| 0.0| 0.0|0.0|
 2| 9| 1| 0.0| 0.0|0.0|
 5| 9| 1| 0.0| 0.0|0.0|
 7| 9| 1| 0.0| 0.0|0.0|
+------------+----+------+---------+------------+---+
only showing top 20 rows

In [81]:
cluster_coef_output = cluster_coef.select('month_passed','user','cc')
cluster_coef_output.sort('user','month_passed').show()

+------------+----+---+
month_passed|user| cc|
+------------+----+---+
 0| 2|0.0|
 0| 3|0.0|
 1| 3|0.0|
 3| 3|0.0|
 4| 3|0.0|
 5| 3|0.0|
 0| 4|0.0|
 1| 4|0.0|
 2| 4|0.0|
 0| 6|0.0|
 10| 6|0.0|
 0| 8|0.0|
 2| 8|0.0|
 7| 8|0.0|
 9| 8|0.0|
 10| 8|0.0|
 0| 9|0.0|
 2| 9|0.0|
 5| 9|0.0|
 7| 9|0.0|
+------------+----+---+
only showing top 20 rows

In [82]:
cluster_coef_output.coalesce(1).write.mode('overwrite').format("parquet").save('/FileStore/tables/cluster_coef_final.parquet')

## 3.3 Pagerank

In [84]:
!pip install networkx
import networkx as nx

In [85]:
#Read file 
# to read parquet file
venmo = spark.read.parquet('/FileStore/tables/VenmoSample_snappy-e020d.parquet')
venmo.show(100)

+--------+-------+----------------+-------------------+--------------------+-----------+--------------------+
 user1| user2|transaction_type| datetime| description|is_business| story_id|
+--------+-------+----------------+-------------------+--------------------+-----------+--------------------+
 1218774|1528945| payment|2015-11-27 10:48:19| Uber| false|5657c473cd03c9af2...|
 5109483|4782303| payment|2015-06-17 11:37:04| Costco| false|5580f9702b64f70ab...|
 4322148|3392963| payment|2015-06-19 07:05:31| Sweaty balls| false|55835ccb1a624b14a...|
 469894|1333620| charge|2016-06-03 23:34:13| 🎥| false|5751b185cd03c9af2...|
 2960727|3442373| payment|2016-05-29 23:23:42| ⚡| false|574b178ecd03c9af2...|
 3977544|2709470| payment|2016-09-29 22:12:07| Chipotlaid| false|57ed2f4723e064eac...|
 3766386|4209061| payment|2016-05-20 10:31:15| kitchen counter| false|573e8503cd03c9af2...|
 730075| 804466| payment|2016-05-26 04:46:45| Food| false|57461d46cd03c9af2...|
 5221751|4993533| payment|2016-07-14 22:53:49| Zaxby| false|5787b58d23e064eac...|
 6843582|7308338| payment|2016-08-31 10:32:46| Fan sucks| false|57c64fdf23e064eac...|
 5317324|3942984| payment|2016-01-04 09:11:25| 👠| false|5689c6bdcd03c9af2...|
 1134661|1556430| payment|2015-10-09 01:53:52| Thanks babe| false|5616bbc0cd03c9af2...|
 4238868|4879587| payment|2015-10-04 08:28:01| 🍺| false|561080a1cd03c9af2...|
11719500|8702716| payment|2016-07-07 21:40:39| ⛽| false|577e69e723e064eac...|
 3625798|5692302| payment|2016-10-16 14:43:41|Hey man it's bee...| false|58032fad23e064eac...|
 613908|3045405| charge|2016-05-07 06:42:17| Getaway car| false|572d2bd9cd03c9af2...|
 4682257|1870271| payment|2016-02-24 09:14:12| 🔮 gypsy things| false|56cd03e4cd03c9af2...|
 9414481|2869012| payment|2016-04-09 09:19:46| 🔴| false|570866c2cd03c9af2...|
 241386|2580543| payment|2015-05-17 06:00:19| Furniture| false|5557cc0407f81c33e...|
 656477| 656214| charge|2013-12-14 22:43:27|bed bath mostly b...| false|52ac6e93d56b6bac5...|
 7077930|2219363| payment|2015-11-03 04:44:39| Fun| false|5637cb37cd03c9af2...|
 219836| 190689| payment|2014-07-17 06:59:52| World Cup Stacks| false|53c711f87d0b0354e...|
 6222232|9644915| payment|2016-10-19 04:55:48|You're lucky we d...| false|58069a6423e064eac...|
 1958142| 252308| charge|2015-05-31 07:33:45| Cable part 2..| false|556a56e94b9f431ba...|
 470775| 558040| payment|2014-07-13 01:15:28| Benny| false|53c17b407d0b0354e...|
 801230|1387123| payment|2014-12-27 07:03:31| Blurb| false|549de943ca134c404...|
 5536694|4679602| payment|2016-09-17 22:54:34| Money| false|57dd673a23e064eac...|
 367859| 384908| charge|2014-01-24 18:14:30| key bees| false|52e23d06c62cecac3...|
 8767419|8458412| payment|2016-06-10 10:59:52| A| false|575a3b38cd03c9af2...|
 2029081| 574846| payment|2014-11-05 06:07:54| Room| false|54594e3a7d0b0354e...|
 3921369|3786234| charge|2016-02-05 14:27:47| 휴지| false|56b440e3cd03c9af2...|
 9212366|7088535| payment|2016-09-28 07:04:15| Mas| false|57eb08ff23e064eac...|
 7533517|6728915| payment|2016-01-24 07:40:37| Last and next week| false|56a40f75cd03c9af2...|
 4725068|5789825| charge|2016-03-01 08:58:44|Cheese squares ak...| false|56d4e944cd03c9af2...|
 2217133|2588336| payment|2015-02-05 19:08:37| Coco| false|54d34f3591bd05aa9...|
 1309265|1594624| payment|2016-02-12 06:22:10| H2o| false|56bd0992cd03c9af2...|
 413268|3521812| payment|2016-04-07 01:16:50|2nd half of SF rent.| false|57055292cd03c9af2...|
 6530112|3087398| payment|2016-02-22 07:02:48|For bills on spli...| false|56ca4218cd03c9af2...|
 2159946| 649658| payment|2015-09-23 18:59:32| Rockies| false|56029424cd03c9af2...|
 2079232|3244153| charge|2015-08-31 05:02:30| Electric| false|55e37d76cd03c9af2...|
 1917492|3124345| payment|2015-11-16 00:41:45| XL| false|5648b5c9cd03c9af2...|
 1391021|1702448| payment|2016-04-03 10:00:11| Spider rolls| false|5700873bcd03c9af2...|
 1556129|2908933| payment|2016-02-09 10:26:11| Nutella| false|56b94e43cd03c9af2...|
 2281104|2221219| payment|2015-06-06 03:05:28| DAT CPK

In [86]:
# to calculate each user's pagerank in each time point, we need to first fetch for both user1 and user2 with the right direction\
# 

venmo.createOrReplaceTempView('venmo')

venmo = spark.sql("""SELECT distinct user1 as user, user2 as friend, datetime, transaction_type
FROM venmo
WHERE transaction_type = 'payment'
UNION 
SELECT distinct user2 as user, user1 as friend, datetime, transaction_type
FROM venmo
WHERE transaction_type = 'charge' """) 


# the result includes all transactions with directed edge 

In [87]:
# create the month indicator for each transaction
window = Window.partitionBy('user').orderBy('datetime')
venmo_with_month= venmo.withColumn("month_passed", F.ceil((F.datediff(venmo.datetime, 
                                   first(venmo.datetime).over(window)))/30))

In [88]:
# loop for i in range(0,13) --- month 0 to 12
schema = StructType([StructField("user", LongType(), True), StructField("pagerank", DoubleType(), True), StructField("month_passed", StringType(), False)])
pagerank_df = sqlContext.createDataFrame([],schema)

for i in range(0,13):
  month_rdd = venmo_with_month.filter(venmo_with_month.month_passed==i).select('user','friend').rdd
  month_tuple_list =month_rdd.map(tuple).collect() 
  G = nx.DiGraph()
  G.add_edges_from(month_tuple_list)
  PR = nx.pagerank_scipy(G, alpha=0.85)
  temp = sc.parallelize([(k,)+(v,) for k,v in PR.items()]).toDF(['user','pagerank'])
  temp = temp.withColumn('month_passed',lit(i))
  pagerank_df = pagerank_df.union(temp)

In [89]:
pagerank_df.coalesce(1).write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/pagerank_df.parquet')

In [90]:
pagerank_df.show()

+------+--------------------+------------+
 user| pagerank|month_passed|
+------+--------------------+------------+
 2866|1.051380846943575...| 0|
 30588|4.261245175419188...| 0|
 6620|1.051380846943575...| 0|
 6507|7.471109503894801E-7| 0|
 20735|4.261245175419188...| 0|
 98941|4.261245175419188...| 0|
 28170|1.068097383237041...| 0|
 27438|7.471109503894801E-7| 0|
 28759|1.051380846943575...| 0|
 65628|4.261245175419188...| 0|
 33602|4.261245175419188...| 0|
137025|4.261245175419188...| 0|
 36525|4.261245175419188...| 0|
 28192|4.261245175419188...| 0|
 43714|1.051380846943575...| 0|
519127|1.068097383237041...| 0|
 45615|4.261245175419188...| 0|
179985|1.710070248932164E-6| 0|
 47283|7.471109503894801E-7| 0|
 50233|7.471109503894801E-7| 0|
+------+--------------------+------------+
only showing top 20 rows

# 4. RF Analyssis

In [92]:
#import package
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import csv
from pyspark.sql.functions import col, create_map, lit
from itertools import chain
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.functions import array, col, explode, struct, lit
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType, IntegerType
from collections import Counter
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import expr

In [93]:
# First, create your dependent variable Y , i.e. the total number of transactions at lifetime point 12. In other words, 
# for every user, you need to count how many transactions s/he had committed during her/his twelve months in Venmo.
venmo = spark.read.parquet('/FileStore/tables/venmo_spending_dyn.parquet')

In [94]:
venmo.createOrReplaceTempView("venmo")
venmo_Y = spark.sql("""SELECT * FROM venmo where month_passed <= 12""")

In [95]:
window = Window.partitionBy('user')
venmo_Y = venmo_Y.select("user","datetime","month_passed").withColumn("Y", F.count(venmo_Y['datetime']).over(window))
venmo_Y = venmo_Y.withColumn("transaction", lit(1))

In [96]:
# Left join the entire month period for each user
venmo.createOrReplaceTempView("Venmo")
month_cycle = spark.sql("""SELECT distinct v.user,sub.month
             FROM Venmo v CROSS JOIN (select 0 as month union all select 1 as month union all select 2 union all select 3 union all select 4 union all select 5 union all select 6 union all select 7 union all select 8 union all select 9 union all select 10 union all select 11 union all select 12) as sub
            ORDER BY v.user,sub.month""")

In [97]:
# Prepare the dataframe structure for calsulating rencency and frequency
venmo_Y = venmo_Y.groupBy('user','month_passed','Y').agg(F.max('datetime').alias('datetime'),F.sum('transaction').alias('transaction'))
venmo_Y.createOrReplaceTempView("venmo_Y")
month_cycle.createOrReplaceTempView("month_cycle")
venmo_Y_recency_prep = spark.sql("""
                SELECT l.user, l.month, v.datetime, v.Y, v.transaction
                FROM month_cycle l LEFT JOIN venmo_Y v
                ON l.user = v.user AND l.month = v.month_passed
                """)

In [98]:
# Filled the null value of datetime
venmo_Y_recency_prep_filled = venmo_Y_recency_prep.withColumn("datetime", func.last('datetime', True).over(Window.partitionBy('user').orderBy('month').rowsBetween(-sys.maxsize, 0)))
# Filled the null value of Y
venmo_Y_recency_prep_filled = venmo_Y_recency_prep_filled.withColumn("Y",F.max(venmo_Y_recency_prep_filled["Y"]).over(Window.partitionBy('user')))
# Create variable indicating the users' first purchasing time
venmo_Y_recency_prep_filled = venmo_Y_recency_prep_filled.withColumn("first_purchase",F.min(venmo_Y_recency_prep_filled["datetime"]).over(Window.partitionBy('user')))
# Create variable indicating whether there exists transaction or not
define_transaction = expr("""IF(transaction IS NULL, 0, transaction)""")
venmo_Y_recency_prep_filled = venmo_Y_recency_prep_filled.withColumn("transaction",define_transaction)
venmo_Y_recency_prep_filled = venmo_Y_recency_prep_filled.cache()

In [99]:
# Create the recency variables.
venmo_Y_recency = venmo_Y_recency_prep_filled.withColumn("recency",30*venmo_Y_recency_prep_filled['month']-F.datediff(venmo_Y_recency_prep_filled['datetime'],venmo_Y_recency_prep_filled['first_purchase']))

In [100]:
# Create the frequency variables.
venmo_Y_recency_frequency = venmo_Y_recency.withColumn("frequency",F.sum(venmo_Y_recency['transaction']).over(Window.partitionBy('user').orderBy('month').rangeBetween(Window.unboundedPreceding,0))/(30*venmo_Y_recency['month']))
# Filled the null value for interval 0 with frequency 1
define_frequency = expr("""IF(frequency IS NULL, 1, frequency)""")
venmo_Y_recency_frequency = venmo_Y_recency_frequency.withColumn("frequency",define_frequency)
venmo_Y_recency_frequency.sort('user','month').show()

+----+-----+-------------------+---+-----------+-------------------+-------+--------------------+
user|month| datetime| Y|transaction| first_purchase|recency| frequency|
+----+-----+-------------------+---+-----------+-------------------+-------+--------------------+
 2| 0|2012-11-23 06:03:42| 1| 1|2012-11-23 06:03:42| 0| 1.0|
 2| 1|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 30| 0.03333333333333333|
 2| 2|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 60|0.016666666666666666|
 2| 3|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 90|0.011111111111111112|
 2| 4|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 120|0.008333333333333333|
 2| 5|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 150|0.006666666666666667|
 2| 6|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 180|0.005555555555555556|
 2| 7|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 210|0.004761904761904762|
 2| 8|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 240|0.004166666666666667|
 2| 9|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 270|0.003703703703703704|
 2| 10|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 300|0.003333333333333...|
 2| 11|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 330|0.003030303030303...|
 2| 12|2012-11-23 06:03:42| 1| 0|2012-11-23 06:03:42| 360|0.002777777777777778|
 3| 0|2016-09-22 15:30:09| 6| 1|2016-09-22 15:30:09| 0| 1.0|
 3| 1|2016-10-09 03:36:13| 6| 5|2016-09-22 15:30:09| 13| 0.2|
 3| 2|2016-10-09 03:36:13| 6| 0|2016-09-22 15:30:09| 43| 0.1|
 3| 3|2016-10-09 03:36:13| 6| 0|2016-09-22 15:30:09| 73| 0.06666666666666667|
 3| 4|2016-10-09 03:36:13| 6| 0|2016-09-22 15:30:09| 103| 0.05|
 3| 5|2016-10-09 03:36:13| 6| 0|2016-09-22 15:30:09| 133| 0.04|
 3| 6|2016-10-09 03:36:13| 6| 0|2016-09-22 15:30:09| 163| 0.03333333333333333|
+----+-----+-------------------+---+-----------+-------------------+-------+--------------------+
only showing top 20 rows

In [101]:
venmo_reg = venmo_Y_recency_frequency.sort('user','month').coalesce(1).write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/venmo_reg.parquet')